In [1]:
import pandas as pd
import numpy as np
from prereise.gather.demanddata.eia import download_map_demand, transform_demand_to_region, fill_missing_demand, find_fix_outliers

In [2]:
#Loadzone to BA map, based on Balancing_Authorities_JAN17.pdf
loadzone_ba = {
    'WA' : ['AVA','BPAT','CHPD','DOPD','GCPD','PSEI','SCL','TPWR'],
    'OR' : ['BPAT','GRID','IPCO','PACW','PGE'],
    'CAnorth' : ['BANC','CISO','PACW','BPAT'],
    'CABayArea' : ['CISO'],
    'CAcentral' : ['CISO', 'TIDC'],
    'CAsw' : ['CISO', 'LDWP'],
    'CAse' : ['CISO', 'IID'],
    'NV' : ['NEVP'],
    'AZ' : ['AZPS','DEAA','GRIF','GRMA','HGMA','PNM','SRP','TEPC','WALC'],
    'UT' : ['PACE'],
    'NM' : ['EPE','PNM'],
    'CO' : ['PSCO','WACM'],
    'WY' : ['PACE','WACM'],
    'ID' : ['AVA','BPAT','IPCO','PACE'],
    'MT' : ['BPAT','GWA','NWMT','WAUW','WWA'],
    'ElPaso' : ['EPE']
}

This codes allow for iteration through nested iterable data structures by returning a generator that can used to create a flattened list

In [ ]:
import collections

# returns generator which recursively iterates through nested lists
def flatten(nestedList):
    for item in nestedList:
        # if this item is iterable, loop through item as well
        # if this item is a string, return the string
        # and don't iterate through the letters
        if isinstance(item, collections.Iterable)\
        and not isinstance(item, (str, bytes)):
            yield from flatten(item)
        else:
            yield item

This flattens out the list of values in the loadzone_ba mapping, returning only the unique values.
BAs are associated with abbreviations. Demand data from EIA is imported using an API Key. Note that data for some BAs
will not be found and this will be reported by the download_map_demand.get_BA_demand function.

In [4]:
western_BAs = list(set(flatten(loadzone_ba.values())))
western_BAs

['GRID',
 'IID',
 'AZPS',
 'HGMA',
 'TEPC',
 'LDWP',
 'WAUW',
 'SCL',
 'PSEI',
 'CHPD',
 'WALC',
 'EPE',
 'GCPD',
 'DOPD',
 'AVA',
 'WACM',
 'TPWR',
 'PNM',
 'DEAA',
 'PSCO',
 'BPAT',
 'GRIF',
 'NWMT',
 'GWA',
 'BANC',
 'WWA',
 'TIDC',
 'GRMA',
 'SRP',
 'PGE',
 'IPCO',
 'CISO',
 'PACE',
 'PACW',
 'NEVP']

In [5]:
start = pd.to_datetime('2016-01-01 00:00:00')
end = pd.to_datetime('2016-12-31 23:00:00')

western_demand = download_map_demand.get_BA_demand(western_BAs, start, end, 'd826fe7862968708eb6a3c2639d6da1b')
western_demand_2016 = western_demand.loc[start:end]
western_demand_2016

GRID not found
HGMA not found
DEAA not found
GRIF not found
GWA not found
WWA not found
GRMA not found


,IID,AZPS,TEPC,LDWP,WAUW,SCL,PSEI,CHPD,WALC,EPE,...,NWMT,BANC,TIDC,SRP,PGE,IPCO,CISO,PACE,PACW,NEVP
2016-01-01 00:00:00+00:00,NaN,2982.0,1624.0,3020.0,NaN,1411.0,4238.0,325.0,838.0,848.0,...,1486.0,1909.0,245.0,2730.0,2887.0,2063.0,23745.0,5703.0,2852.0,3910.0
2016-01-01 01:00:00+00:00,NaN,3363.0,1801.0,3146.0,NaN,1488.0,4712.0,353.0,904.0,948.0,...,1588.0,2028.0,256.0,3026.0,3032.0,2248.0,24213.0,6170.0,3036.0,4220.0
2016-01-01 02:00:00+00:00,NaN,3642.0,1788.0,3518.0,NaN,1588.0,5085.0,371.0,948.0,988.0,...,1597.0,2271.0,284.0,3262.0,3285.0,2328.0,25174.0,6274.0,3296.0,4593.0
2016-01-01 03:00:00+00:00,NaN,3615.0,1763.0,3547.0,NaN,1551.0,5030.0,366.0,937.0,959.0,...,1553.0,2301.0,285.0,3220.0,3233.0,2289.0,28423.0,5914.0,3290.0,4607.0
2016-01-01 04:00:00+00:00,NaN,3568.0,1739.0,3452.0,NaN,1484.0,4845.0,365.0,922.0,929.0,...,1516.0,2242.0,276.0,3142.0,3099.0,2249.0,28784.0,6136.0,3189.0,4523.0
2016-01-01 05:00:00+00:00,NaN,3511.0,1689.0,3341.0,NaN,1421.0,4581.0,362.0,911.0,893.0,...,1500.0,2176.0,266.0,3098.0,2990.0,2203.0,27997.0,5902.0,3099.0,4420.0
2016-01-01 06:00:00+00:00,NaN,3403.0,1640.0,3199.0,NaN,1358.0,4419.0,350.0,889.0,853.0,...,1448.0,2115.0,257.0,3038.0,2824.0,2130.0,27033.0,5798.0,2982.0,4299.0
2016-01-01 07:00:00+00:00,NaN,3303.0,1602.0,3055.0,NaN,1286.0,4200.0,339.0,927.0,820.0,...,1398.0,2006.0,246.0,2965.0,2663.0,2073.0,25967.0,5580.0,2835.0,4189.0
2016-01-01 08:00:00+00:00,NaN,3235.0,1569.0,2922.0,NaN,1240.0,4065.0,335.0,954.0,792.0,...,1361.0,1922.0,237.0,2935.0,2553.0,2040.0,24859.0,5458.0,2750.0,4129.0
2016-01-01 09:00:00+00:00,258.0,3230.0,1580.0,2814.0,NaN,1197.0,3931.0,333.0,957.0,768.0,...,1335.0,1849.0,230.0,2927.0,2459.0,2021.0,24036.0,5169.0,2677.0,4028.0


In [6]:
filled_western_demand_2016 = fill_missing_demand.replace_with_shifted_demand(western_demand_2016, start, end)

C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\prereise\gather\demanddata\eia\fill_missing_demand.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  shifted_demand_BA[baName] = fill_ba_demand(shifted_demand_BA, baName, dayMap)
C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\prereise\gather\demanddata\eia\fill_missing_demand.py:31: Se

In [7]:
for baName in filled_western_demand_2016.columns:
    print(baName)
    print(f'Num missing values: {filled_western_demand_2016.loc[filled_western_demand_2016[baName].isnull()].shape[0]}')
    print()
    

IID
Num missing values: 0

AZPS
Num missing values: 0

TEPC
Num missing values: 0

LDWP
Num missing values: 0

WAUW
Num missing values: 50

SCL
Num missing values: 0

PSEI
Num missing values: 0

CHPD
Num missing values: 0

WALC
Num missing values: 0

EPE
Num missing values: 0

GCPD
Num missing values: 0

DOPD
Num missing values: 365

AVA
Num missing values: 0

WACM
Num missing values: 0

TPWR
Num missing values: 0

PNM
Num missing values: 0

PSCO
Num missing values: 0

BPAT
Num missing values: 0

NWMT
Num missing values: 0

BANC
Num missing values: 0

TIDC
Num missing values: 0

SRP
Num missing values: 0

PGE
Num missing values: 0

IPCO
Num missing values: 0

CISO
Num missing values: 0

PACE
Num missing values: 16

PACW
Num missing values: 40

NEVP
Num missing values: 0



In [ ]:
Some BAs are very small. In these cases, BAs are agregated into BA areas as follows:   

In [8]:
western_agg_demand = transform_demand_to_region.transform_ba_to_region(filled_western_demand_2016, loadzone_ba)
western_agg_demand

WA regional demand was summed from ['BPAT', 'PSEI', 'SCL', 'CHPD', 'GCPD', 'DOPD', 'AVA', 'TPWR']


******************************
Missing BA columns for OR!
Original columns: ['BPAT', 'GRID', 'IPCO', 'PACW', 'PGE']
******************************
OR regional demand was summed from ['BPAT', 'PACW', 'IPCO', 'PGE']

CAnorth regional demand was summed from ['BPAT', 'BANC', 'PACW', 'CISO']

CABayArea regional demand was summed from ['CISO']

CAcentral regional demand was summed from ['TIDC', 'CISO']

CAsw regional demand was summed from ['LDWP', 'CISO']

CAse regional demand was summed from ['IID', 'CISO']

NV regional demand was summed from ['NEVP']


******************************
Missing BA columns for AZ!
Original columns: ['AZPS', 'DEAA', 'GRIF', 'GRMA', 'HGMA', 'PNM', 'SRP', 'TEPC', 'WALC']
******************************
AZ regional demand was summed from ['AZPS', 'TEPC', 'WALC', 'SRP', 'PNM']

UT regional demand was summed from ['PACE']

NM regional demand was summed from ['PNM', 'EP

,WA,OR,CAnorth,CABayArea,CAcentral,CAsw,CAse,NV,AZ,UT,NM,CO,WY,ID,MT,ElPaso
2016-01-01 00:00:00+00:00,16980.0,15491.0,36195.0,23745.0,23990.0,26765.0,24011.0,3910.0,9890.0,5703.0,2564.0,9063.0,8957.0,17244.0,9296.0,848.0
2016-01-01 01:00:00+00:00,18265.0,16459.0,37420.0,24213.0,24469.0,27359.0,24500.0,4220.0,11005.0,6170.0,2859.0,9817.0,9642.0,18491.0,9855.0,948.0
2016-01-01 02:00:00+00:00,19398.0,17543.0,39375.0,25174.0,25458.0,28692.0,25494.0,4593.0,11607.0,6274.0,2955.0,9760.0,9722.0,19227.0,10356.0,988.0
2016-01-01 03:00:00+00:00,19239.0,17410.0,42612.0,28423.0,28708.0,31970.0,28752.0,4607.0,11475.0,5914.0,2899.0,9529.0,9331.0,18771.0,10272.0,959.0
2016-01-01 04:00:00+00:00,18692.0,16927.0,42605.0,28784.0,29060.0,32236.0,29108.0,4523.0,11293.0,6136.0,2851.0,9306.0,9510.0,18680.0,10024.0,929.0
2016-01-01 05:00:00+00:00,18087.0,16460.0,41440.0,27997.0,28263.0,31338.0,28311.0,4420.0,11096.0,5902.0,2780.0,9005.0,9184.0,18146.0,9786.0,893.0
2016-01-01 06:00:00+00:00,17511.0,15865.0,40059.0,27033.0,27290.0,30232.0,27328.0,4299.0,10798.0,5798.0,2681.0,8688.0,9028.0,17672.0,9487.0,853.0
2016-01-01 07:00:00+00:00,16797.0,15205.0,38442.0,25967.0,26213.0,29022.0,26231.0,4189.0,10565.0,5580.0,2588.0,8465.0,8789.0,17040.0,9140.0,820.0
2016-01-01 08:00:00+00:00,16419.0,15099.0,37287.0,24859.0,25096.0,27781.0,25096.0,4129.0,10398.0,5458.0,2497.0,8273.0,8662.0,16969.0,9220.0,792.0
2016-01-01 09:00:00+00:00,16000.0,14450.0,35855.0,24036.0,24266.0,26850.0,24294.0,4028.0,10349.0,5169.0,2423.0,8076.0,8308.0,16171.0,8727.0,768.0


In [ ]:
Check for the presence of NaNs in the dataset:   

In [9]:
for baName in western_agg_demand.columns:
    print(baName)
    print(f'Num missing values: {western_agg_demand.loc[western_agg_demand[baName].isnull()].shape[0]}')
    print()

WA
Num missing values: 0

OR
Num missing values: 0

CAnorth
Num missing values: 0

CABayArea
Num missing values: 0

CAcentral
Num missing values: 0

CAsw
Num missing values: 0

CAse
Num missing values: 0

NV
Num missing values: 0

AZ
Num missing values: 0

UT
Num missing values: 16

NM
Num missing values: 0

CO
Num missing values: 0

WY
Num missing values: 0

ID
Num missing values: 0

MT
Num missing values: 0

ElPaso
Num missing values: 0



Use adjacent demand data to fill missing values using a series of rules:
1. Monday: look forward one day
2. Tues - Thurs: average of look forward one day and look back one day
3. Fri: look back one day
4. Sat: look forward one day
5. Sun: look back one day

If data is still missing after applying the above rules, week ahead and week behind data is used
1. Monday: look forward two days 
2. Tues: look forward two days
3. Wed: average of look forward two days and look back two days
4. Thurs: look back two days
5. Fri: look back two days
6. Sat - Sun: average of look back one week and look forward one week

If data is still missing after applying the above rules, week ahead and week behind data is used:
1. Mon - Sun: average of look back one week and look forward one week

In [ ]:
western_fix_outliers = find_fix_outliers.fix_dataframe_outliers(western_agg_demand)

In [13]:
ba_loadzone_frac = defaultdict(int)
for ba,counties in ba_loadzone_county_split.items():
    for county in counties:
        c = county.split('_')
#         print(c)
        if c[0] == 'Dona Ana':
            c[0] = 'Doña Ana'
        ba_loadzone_frac[ba] += int(counties_pop.loc[(counties_pop['County']==c[0]) & (counties_pop['State']==c[1].upper())].iloc[0]['pop2016'])
ba_total = defaultdict(int)
for ba,pop in ba_loadzone_frac.items():
    ba_total[ba.split('_')[0]] += pop
for ba in ba_loadzone_frac:
    name = ba.split('_')[0]
    ba_loadzone_frac[ba] /= ba_total[name]
ba_loadzone_frac

NameError: name 'defaultdict' is not defined

In [ ]:
#Get inaccurate BA county table
dir_files = '../test/data/'

county_utility_file = 'Service_Territory_2016.xlsx'  #get utility-county mapping
ba_utility_file = 'Sales_Ult_Cust_2016.xlsx'  #Get utility-BA mapping
# ba_list_file = 'Balancing_Authority_2016.xlsx'  #list of BAs by state

county_utility =  pd.read_excel(io = dir_files + '/' + county_utility_file, 
                                   dtype = {'Utility Number': str, 'County': str, 'State': str}
                               )
ba_utility = pd.read_excel(io = dir_files + '/' + ba_utility_file, 
                          usecols = 'B,G,I,W', skiprows = [0,1], 
                          dtype = {'Utility Number': str, 'BA_CODE': str, 'State': str},
                          sheet_name = 'States'
                          )
ba_county = pd.merge(county_utility, ba_utility, how='outer', on ='Utility Number')

#Cleaning
ba_county_uniq = ba_county[['BA_CODE','County','State_x','State_y']].drop_duplicates(keep='first')
ba_county_uniq.drop(columns = ['State_y'], inplace=True)
ba_county_uniq.rename(columns = {'State_x':'State'}, inplace=True)
ba_county_uniq.dropna(axis = 0, how = 'any', inplace=True)
ba_county_uniq.drop_duplicates(keep = 'first', inplace=True)

In [ ]:
AVA_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='AVA']
BPAT_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='BPAT']
IPCO_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='IPCO']
PACW_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='PACW']
PNM_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='PNM']
PACE_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='PACE']
EPE_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='EPE']
WACM_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='WACM']

# NEVP_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='NEVP']
# WALC_county = ba_county_uniq[ba_county_uniq['BA_CODE']=='WALC']

In [ ]:
us_counties_pop_file = 'USCountyPop.csv'
counties_pop = pd.read_csv(dir_files + '/' + us_counties_pop_file, header=0)
#cleaning
counties_pop.drop(columns = ['GEO.id', 'GEO.id2', 'rescen42010', 'resbase42010',
       'respop72010', 'respop72011', 'respop72012', 'respop72013',
       'respop72014', 'respop72015','respop72017'], inplace = True)
counties_pop['County'] = counties_pop['GEO.display-label'].str.split(',',expand=True)[0].str.replace(' County','')
counties_pop['State'] = counties_pop['GEO.display-label'].str.split(',',expand=True)[1].str.replace('^ ','')
counties_pop['State'] = counties_pop['State'].apply(lambda x: us_state_abbrev[x])
counties_pop.drop(columns = 'GEO.display-label', inplace = True)
counties_pop.rename(columns = {'respop72016':'pop2016'}, inplace=True)

In [ ]:
target_states = {'WA','ID'}
AVA_county_filtered_by_loadzone = set()
for index, row in AVA_county.iterrows():
    if row['State'] in target_states:
        AVA_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'WA','ID','MT','OR','CA'}
# target_states = us_state_abbrev
BPAT_county_filtered_by_loadzone = set()
for index, row in BPAT_county.iterrows():
    if row['State'] in target_states:
        BPAT_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'ID','OR'}
IPCO_county_filtered_by_loadzone = set()
for index, row in IPCO_county.iterrows():
    if row['State'] in target_states:
        IPCO_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'CA','OR'}
# target_states = us_state_abbrev
PACW_county_filtered_by_loadzone = set()
for index, row in PACW_county.iterrows():
    if row['State'] in target_states:
        PACW_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'AZ','NM'}
PNM_county_filtered_by_loadzone = set()
for index, row in PNM_county.iterrows():
    if row['State'] in target_states:
        PNM_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'ID','UT','WY'}
# target_states = us_state_abbrev
PACE_county_filtered_by_loadzone = set()
for index, row in PACE_county.iterrows():
    if row['State'] in target_states:
        PACE_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

target_states = {'NM','TX'}
EPE_county_filtered_by_loadzone = set()
for index, row in EPE_county.iterrows():
    if row['State'] in target_states:
        EPE_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'CO','WY'}
WACM_county_filtered_by_loadzone = set()
for index, row in WACM_county.iterrows():
    if row['State'] in target_states:
        WACM_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

# target_states = us_state_abbrev
# NEVP_county_filtered_by_loadzone = set()
# for index, row in NEVP_county.iterrows():
#     if row['State'] in target_states:
#         NEVP_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
# target_states = us_state_abbrev
# WALC_county_filtered_by_loadzone = set()
# for index, row in WALC_county.iterrows():
#     if row['State'] in target_states:
#         WALC_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

In [ ]:
ba_loadzone_frac = defaultdict(int)
for ba,counties in ba_loadzone_county_split.items():
    for county in counties:
        c = county.split('_')
#         print(c)
        if c[0] == 'Dona Ana':
            c[0] = 'Doña Ana'
        ba_loadzone_frac[ba] += int(counties_pop.loc[(counties_pop['County']==c[0]) & (counties_pop['State']==c[1].upper())].iloc[0]['pop2016'])
ba_total = defaultdict(int)
for ba,pop in ba_loadzone_frac.items():
    ba_total[ba.split('_')[0]] += pop
for ba in ba_loadzone_frac:
    name = ba.split('_')[0]
    ba_loadzone_frac[ba] /= ba_total[name]
ba_loadzone_frac

In [ ]:
target_states = {'WA','ID'}
AVA_county_filtered_by_loadzone = set()
for index, row in AVA_county.iterrows():
    if row['State'] in target_states:
        AVA_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'WA','ID','MT','OR','CA'}
# target_states = us_state_abbrev
BPAT_county_filtered_by_loadzone = set()
for index, row in BPAT_county.iterrows():
    if row['State'] in target_states:
        BPAT_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'ID','OR'}
IPCO_county_filtered_by_loadzone = set()
for index, row in IPCO_county.iterrows():
    if row['State'] in target_states:
        IPCO_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'CA','OR'}
# target_states = us_state_abbrev
PACW_county_filtered_by_loadzone = set()
for index, row in PACW_county.iterrows():
    if row['State'] in target_states:
        PACW_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'AZ','NM'}
PNM_county_filtered_by_loadzone = set()
for index, row in PNM_county.iterrows():
    if row['State'] in target_states:
        PNM_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'ID','UT','WY'}
# target_states = us_state_abbrev
PACE_county_filtered_by_loadzone = set()
for index, row in PACE_county.iterrows():
    if row['State'] in target_states:
        PACE_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

target_states = {'NM','TX'}
EPE_county_filtered_by_loadzone = set()
for index, row in EPE_county.iterrows():
    if row['State'] in target_states:
        EPE_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
target_states = {'CO','WY'}
WACM_county_filtered_by_loadzone = set()
for index, row in WACM_county.iterrows():
    if row['State'] in target_states:
        WACM_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

# target_states = us_state_abbrev
# NEVP_county_filtered_by_loadzone = set()
# for index, row in NEVP_county.iterrows():
#     if row['State'] in target_states:
#         NEVP_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())
        
# target_states = us_state_abbrev
# WALC_county_filtered_by_loadzone = set()
# for index, row in WALC_county.iterrows():
#     if row['State'] in target_states:
#         WALC_county_filtered_by_loadzone.add(str(row['County'])+'_'+str(row['State']).lower())

In [ ]:
from collections import defaultdict

ba_loadzone_county_split = defaultdict(list)

#AVA################################################################
ba_loadzone_county_split['AVA_WA'] = ['Ferry_wa', 'Stevens_wa', 'Pend Oreille_wa','Lincoln_wa','Spokane_wa','Whitman_wa']
ba_loadzone_county_split['AVA_ID'] = ['Bonner_id','Kootenai_id','Benewah_id','Latah_id']    
#BPAT###############################################################
#Note: the region of SCL is less than a county, IGNORED here, keep King County in BPAT_WA
GCPD = ['Grant_wa']
DOPD = ['Okanogan_wa','Douglas_wa']
CHPD = ['Chelan_wa']
PSEI = ['Whatcom_wa','Skagit_wa']
TPWR = ['Thurston_wa']
AVA_WA = ['Ferry_wa', 'Stevens_wa', 'Pend Oreille_wa','Lincoln_wa','Spokane_wa','Whitman_wa']
for county in BPAT_county_filtered_by_loadzone:
    if county[-2:] == 'wa' and county not in set(GCPD+DOPD+CHPD+PSEI+TPWR+AVA_WA):
        ba_loadzone_county_split['BPAT_WA'].append(county)
ba_loadzone_county_split['BPAT_WA'].append('King_wa')

ba_loadzone_county_split['BPAT_CAnorth'] = ['Modoc_ca']
ba_loadzone_county_split['BPAT_ID'] = ['Shoshone_id','Clearwater_id','Lewis_id','Nez Perce_id','Boundary_id']
ba_loadzone_county_split['BPAT_MT'] = ['Lincoln_mt','Flathead_mt','Lake_mt','Sanders_mt','Mineral_mt']

PACW_OR = ['Douglas_or','Coos_or','Josephine_or','Jackson_or','Klamath_or','Curry_or']
PGE = ['Washington_or','Yamhill_or','Polk_or','Marion_or','Clackamas_or','Multnomah_or','Hood River_or']
GRID = ['Morrow_or']
for county in BPAT_county_filtered_by_loadzone:
    if county[-2:] == 'or' and county not in set(PGE+GRID+PACW_OR):
        ba_loadzone_county_split['BPAT_OR'].append(county)

#IPCO###############################################################
ba_loadzone_county_split['IPCO_OR'] = ['Malheur_or']
for county in IPCO_county_filtered_by_loadzone:
    if county[-2:] == 'id':
        ba_loadzone_county_split['IPCO_ID'].append(county)
ba_loadzone_county_split['IPCO_ID'].append('Butte_id')
ba_loadzone_county_split['IPCO_ID'].remove('Oneida_id')
ba_loadzone_county_split['IPCO_ID'].remove('Bannock_id')
ba_loadzone_county_split['IPCO_ID'].remove('Power_id')
ba_loadzone_county_split['IPCO_ID'].remove('Bingham_id')

#PACW################################################################
ba_loadzone_county_split['PACW_OR'] = ['Douglas_or','Coos_or','Josephine_or','Jackson_or','Klamath_or','Curry_or']
ba_loadzone_county_split['PACW_CAnorth'] = ['Del Norte_ca','Siskiyou_ca']

#PNM#################################################################
ba_loadzone_county_split['PNM_AZ'].append('Greenlee_az')
for county in PNM_county_filtered_by_loadzone:
    if county[-2:] == 'nm':
        ba_loadzone_county_split['PNM_NM'].append(county)
ba_loadzone_county_split['PNM_NM'].append('Lea_nm')
ba_loadzone_county_split['PNM_NM'].append('Roosevelt_nm')
ba_loadzone_county_split['PNM_NM'].append('Curry_nm')
ba_loadzone_county_split['PNM_NM'].remove('Dona Ana_nm')
ba_loadzone_county_split['PNM_NM'].remove('Otero_nm')

#PACE################################################################
for county in PACE_county_filtered_by_loadzone:
    if county[-2:] == 'ut':
        ba_loadzone_county_split['PACE_UT'].append(county)
ba_loadzone_county_split['PACE_UT'].append('Kane_ut')
ba_loadzone_county_split['PACE_UT'].append('Wayne_ut')
ba_loadzone_county_split['PACE_UT'].append('Daggett_ut')

for county in PACE_county_filtered_by_loadzone:
    if county[-2:] == 'wy':
        ba_loadzone_county_split['PACE_WY'].append(county)
ba_loadzone_county_split['PACE_WY'].remove('Albany_wy')
ba_loadzone_county_split['PACE_WY'].remove('Platte_wy')
ba_loadzone_county_split['PACE_WY'].remove('Converse_wy')
ba_loadzone_county_split['PACE_WY'].remove('Natrona_wy')
ba_loadzone_county_split['PACE_WY'].remove('Johnson_wy')
ba_loadzone_county_split['PACE_WY'].append('Teton_wy')

for county in PACE_county_filtered_by_loadzone:
    if county[-2:] == 'id':
        ba_loadzone_county_split['PACE_ID'].append(county)
ba_loadzone_county_split['PACE_ID'].remove('Butte_id')

#WACM################################################################
for county in WACM_county_filtered_by_loadzone:
    if county[-2:] == 'wy':
        if county not in ba_loadzone_county_split['PACE_WY']:
            ba_loadzone_county_split['WACM_WY'].append(county)
# del ba_loadzone_county_split['PACE_WY']

PSCO = ['Alamosa_co','Costilla_co','Huerfano_co','Las Animas_co','Baca_co','Prowers_co','Bent_co','Otero_co','Pueblo_co','Crowley_co','Lincoln_co','El Paso_co','Teller_co','Douglas_co','Elbert_co','Jefferson_co','Denver_co','Arapahoe_co','Adams_co','Clear Creek_co','Gilpin_co','Boulder_co','Weld_co','Morgan_co','Broomfield_co']
for county in WACM_county_filtered_by_loadzone:
    if county[-2:] == 'co':
        if county not in PSCO:
            ba_loadzone_county_split['WACM_CO'].append(county)
ba_loadzone_county_split['WACM_CO'].append('Eagle_co')
ba_loadzone_county_split['WACM_CO'].append('Park_co')

#EPE#################################################################
ba_loadzone_county_split['EPE_NM'] = ['Dona Ana_nm','Otero_nm','Eddy_nm']
ba_loadzone_county_split['EPE_ElPaso'] = ['El Paso_tx','Hudspeth_tx','Culberson_tx']

#CISO################################################################
# 'CISO': {'CABayArea', 'CAcentral', 'CAnorth', 'CAse', 'CAsw'}
    
ba_loadzone_county_split['CISO_CAnorth'] = ['Butte','Colusa','Del Norte','El Dorado','Glenn','Humboldt',
                       'Lake','Lassen','Mendocino','Modoc','Nevada','Placer','Plumas',
                       'Sacramento','Shasta','Sierra','Siskiyou','Sutter','Tehama',
                       'Trinity','Yolo','Yuba']
ba_loadzone_county_split['CISO_CABayArea'] = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo',
            'Santa Clara','Santa Cruz','Solano','Sonoma']
ba_loadzone_county_split['CISO_CAcentral'] = ['Alpine','Amador','Calaveras','Fresno','Inyo','Kings','Madera',
                      'Mariposa','Merced','Mono','Monterey','San Benito','San Joaquin',
                      'Stanislaus','Tulare','Tuolumne']
ba_loadzone_county_split['CISO_CAse'] = ['Imperial','Orange','Riverside','San Bernardino','San Diego']
ba_loadzone_county_split['CISO_CAsw'] = ['Kern','Los Angeles','San Luis Obispo','Santa Barbara','Ventura']

for ba_loadzone in ba_loadzone_county_split:
    if ba_loadzone.split('_')[0] == 'CISO':
        for i,county in enumerate(ba_loadzone_county_split[ba_loadzone]):
            ba_loadzone_county_split[ba_loadzone][i] = county+'_'+'ca'
# ba_loadzone_county_split